## Preprocès: ##

- Eliminar els digits del text
- Convertir tot el text a minúscula
- Substitueix els espais en blanc continus per un de sol
- Concatena totes les frases amb un espai doble al mig 


In [2]:
import re 
from nltk.collocations import TrigramCollocationFinder, ngrams
from collections import Counter, defaultdict
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
def preprocess_text(text):
    text = re.sub(r'\d+', '', text) 
    text = text.lower()  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\n', '  ', text)  
    return text

def load_and_preprocess_data(data_path):
    with open(data_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return preprocess_text(text)


languages = ['deu', 'eng', 'fra', 'ita', 'nld', 'spa']
path = 'C:/Users/Paess/Documents/Uni/Segon/PLH/langId'
train = {}
test = []

for idiom in languages:
    file_path = os.path.join(path, f'{idiom}_trn.txt')
    text = load_and_preprocess_data(file_path)
    train[idiom] = text


X_test = []
y_test = []

for idiom in languages:
    file_path = os.path.join(path, f'{idiom}_tst.txt')

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines() 
    
    for line in lines:
        sentence = preprocess_text(line.strip()) 
        if sentence:  
            X_test.append(sentence)
            y_test.append(idiom)

train_split = {}
val_split = {}

for idiom in languages:
    text = train[idiom]
    split_index = int(len(text) * 0.8)
    train_split[idiom] = text[:split_index]
    val_split[idiom] = text[split_index:]



In [9]:
def generate_trigrams(text):
    
    lletres = list(text)  # Convierte el texto en una lista de caracteres
    trigrams = [''.join(trigram) for trigram in ngrams(lletres, 3) if len(trigram) == 3]
    return trigrams

def frequent_trigrams(trigrams):
    trigram_counts = Counter(trigrams)
    frequent_trigrams_set = {trigram for trigram, count in trigram_counts.items() if count >= 5}
    return [trigram for trigram in trigrams if trigram in frequent_trigrams_set]



for idiom in train_split: 
    train_split[idiom] = frequent_trigrams(generate_trigrams(train_split[idiom]))
    val_split[idiom] = frequent_trigrams(generate_trigrams(val_split[idiom]))
    


In [10]:
info_lang = {}
for lang, trigrams in train_split.items():
    NT = len(trigrams)
    B = 30**3
    info_lang[lang] = {'NT': NT, 'B': B}

LID = Ct(ej)+lambda/Nt+lambdaB, B= # trigrames diferents Nt= # trigrames 

In [11]:
def LID(count_trigram, NT, B, lam=0.5):
    result= np.log((count_trigram + lam) / (NT + lam * B))
    return result


In [12]:
def entrenament_model(train, lam=0.5):
    trigram_counts = {trigram: {lang: 0 for lang in languages} for lang in languages for trigram in train.get(lang, [])}
    
    for lang, lang_trigrams in train.items():
        for trigram in lang_trigrams:
            trigram_counts[trigram][lang] += 1
    
    trigram_LID = {}
    for trigram, lang_counts in trigram_counts.items():
        trigram_LID[trigram] = {lang: LID(count, info_lang[lang]['NT'], info_lang[lang]['B'], lam) for lang, count in lang_counts.items()}
    
    return trigram_LID

In [ ]:

def predict(text, trigram_LID, lam=0.5):
    text_preprocessat = preprocess_text(text)  
    trigrams = generate_trigrams(text_preprocessat) 
    
    LID_per_lang = {lang: 0 for lang in info_lang}  
    
    for trigram in trigrams:
        if trigram in trigram_LID:
            LID_per_lang_trigram = trigram_LID[trigram]  
        else:
            LID_per_lang_trigram = {lang: LID(0, info_lang[lang]['NT'], info_lang[lang]['B'], lam) for lang in info_lang}
        for lang, LID_trigram in LID_per_lang_trigram.items():
            LID_per_lang[lang] += LID_trigram 

    min_val = min(LID_per_lang.values())
    max_val = max(LID_per_lang.values())
    
    if max_val == min_val: 
        probabilities = {lang: float(100 / len(LID_per_lang)) for lang in LID_per_lang}
    else:
        probabilities = {lang: float(((score - min_val) / (max_val - min_val)) * 100) for lang, score in LID_per_lang.items()}  

    max_language = max(probabilities, key=probabilities.get)  

    return probabilities, max_language

In [14]:
trigram_LID = entrenament_model(train_split, 0.5)


In [15]:
text = "Hola, com estás?"

print(predict(text, trigram_LID, 0.5))

({'deu': 0.0, 'eng': 35.04509486508014, 'fra': 35.185960429637674, 'ita': 44.41237071387217, 'nld': 5.242111175603981, 'spa': 100.0}, 'spa')


In [16]:
# Function to evaluate the results of the LID system by comparing the predicted languages with the actual languages

def test(X_test, y_test, trigram_LID, lam=0.5):
    y_pred = []
    for sentence, actual_lang in zip(X_test, y_test):
        _, lang = predict(sentence, trigram_LID, lam)
        y_pred.append(lang)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average=None)
    f1 = f1_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, precision, f1, conf_matrix

In [ ]:
accuracy, precision, f1, conf_matrix = test(X_test, y_test, trigram_LID, 0.5)

# Mostrar métricas de evaluación
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision}')
print(f'F1-score: {f1:.4f}')

# Define the class labels
classes = np.unique(y_test)


# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 12}, cmap="Blues", fmt=".2f", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
